<h3>Data</h3>

http://www.abs.gov.au/ausstats/abs@.nsf/mf/1800.0

<h3>Dependencies</h3>

In [4]:
import pandas as pd
import numpy as np
import re

<h3>DataFrame Preprocessing</h3>

In [5]:
df = pd.read_excel('poll.xls', sheet_name='Table 5', skiprows=5, nrows=630) \
                    .rename({'Unnamed: 0':'Area', 'Unnamed: 1':'Measure', \
                       'Total Males(b)':'Total for gender'}, axis='columns')

#We'd like to change South Division (b) --> South Division
PATTERN = "\\([a-z]\\)"
df.replace(PATTERN, '', regex=True, inplace=True)
#Get rid of sum ups
df['State'] = np.where(df['Area'].str.contains('|'.join(['Australia', 'Divisions']), na=False), df['Area'], None)
df.State.ffill(inplace=True)
#Handle merged cells
df.Area.fillna(method='ffill', limit=2, inplace=True)
df = df[~df.Area.str.contains('|'.join(['Australia', 'Divisions']), na=False)]
df.dropna(inplace=True)
#Redundant words
df.replace('Divisions', '', regex=True, inplace=True)
df.head(10)

,Area,Measure,18-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85 years and over,Total for gender,State
1,Banks,Total participants,1102.0,3098.0,2918.0,2895.0,2982.0,3172.0,3477.0,3523.0,3764.0,3297.0,2851.0,2366.0,1636.0,1352.0,1247.0,39680.0,New South Wales
2,Banks,Eligible participants,1431.0,4224.0,4036.0,4188.0,4197.0,4239.0,4419.0,4471.0,4621.0,3948.0,3257.0,2631.0,1836.0,1531.0,1499.0,50528.0,New South Wales
3,Banks,Participation rate (%),77.0,73.3,72.3,69.1,71.1,74.8,78.7,78.8,81.5,83.5,87.5,89.9,89.1,88.3,83.2,78.5,New South Wales
5,Barton,Total participants,977.0,3047.0,3231.0,3678.0,3667.0,3673.0,3744.0,3610.0,3291.0,2948.0,2720.0,2293.0,1775.0,1323.0,1111.0,41088.0,New South Wales
6,Barton,Eligible participants,1278.0,4186.0,4594.0,5305.0,5176.0,4935.0,4852.0,4562.0,4162.0,3670.0,3245.0,2685.0,1999.0,1599.0,1368.0,53616.0,New South Wales
7,Barton,Participation rate (%),76.4,72.8,70.3,69.3,70.8,74.4,77.2,79.1,79.1,80.3,83.8,85.4,88.8,82.7,81.2,76.6,New South Wales
9,Bennelong,Total participants,1177.0,3356.0,3106.0,3245.0,3404.0,3405.0,3563.0,3742.0,3657.0,3301.0,2863.0,2124.0,1620.0,1221.0,1195.0,40979.0,New South Wales
10,Bennelong,Eligible participants,1488.0,4492.0,4316.0,4580.0,4792.0,4463.0,4456.0,4586.0,4367.0,3830.0,3239.0,2351.0,1785.0,1371.0,1422.0,51538.0,New South Wales
11,Bennelong,Participation rate (%),79.1,74.7,72.0,70.9,71.0,76.3,80.0,81.6,83.7,86.2,88.4,90.3,90.8,89.1,84.0,79.5,New South Wales
13,Berowra,Total participants,1523.0,3767.0,2687.0,2241.0,2594.0,3439.0,3852.0,4073.0,4209.0,3736.0,3235.0,2799.0,1841.0,1212.0,1085.0,42293.0,New South Wales


<h3>Pivoting for age count</h3>

In [6]:
age_count = pd.melt(df, id_vars=('State', 'Area', 'Measure'), var_name='Age', value_name='Count')
age_count['Gender'] = pd.Series('Male', index=age_count.index)
age_count = age_count[['State', 'Area', 'Measure', 'Gender', 'Age', 'Count']]
age_count.head(10)

,State,Area,Measure,Gender,Age,Count
0,New South Wales,Banks,Total participants,Male,18-19 years,1102.0
1,New South Wales,Banks,Eligible participants,Male,18-19 years,1431.0
2,New South Wales,Banks,Participation rate (%),Male,18-19 years,77.0
3,New South Wales,Barton,Total participants,Male,18-19 years,977.0
4,New South Wales,Barton,Eligible participants,Male,18-19 years,1278.0
5,New South Wales,Barton,Participation rate (%),Male,18-19 years,76.4
6,New South Wales,Bennelong,Total participants,Male,18-19 years,1177.0
7,New South Wales,Bennelong,Eligible participants,Male,18-19 years,1488.0
8,New South Wales,Bennelong,Participation rate (%),Male,18-19 years,79.1
9,New South Wales,Berowra,Total participants,Male,18-19 years,1523.0
